In [1]:
# ensure packages when running in a notebook are always reloaded
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
import matplotlib.colors as mcolors
import numpy as np

In [3]:
import sys
sys.path.append("..")

from viz.plot_token_rewards import plot_token_rewards

In [4]:
tokenizer = AutoTokenizer.from_pretrained('reshinthadith/codegen_350M_list_manip_5_len')

In [101]:
# def compare_reward_funcs(reward_funcs: List):
#     input_outputs = [
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4]),1)"],
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -a]),1)"],
#         ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -3]),1)"],
#     ]
#     samples = [f"{input_output[0]} {input_output[1]}" for input_output in input_outputs]
#     for reward_func in reward_funcs:
#         rewards_per_sample = reward_func(samples)
#         # for sample, reward in zip(samples, rewards_per_sample):
#         for sample, input, output, reward in zip(samples, [input_output[0] for input_output in input_outputs], [input_output[1] for input_output in input_outputs], rewards_per_sample):
#             # print(f"{reward_func.__name__}: ({sample}) = {reward}")
#             # title = f"{reward_func.__name__}:\n{input} = {reward}"
#             # plotly title with newline
#             title = f"{reward_func.__name__} | {reward}<br>Prompt: {input}<br>Correct: {output}"
#             print(title)

#             tokens = tokenizer.tokenize(output)
#             plot_token_rewards(tokens, reward, title=title)

#         # plot_token_rewards(samples, rewards)

In [61]:
# EOS_TOKEN = "<|endoftext|>"
EOS_TOKEN = ""

def compare_reward_funcs(reward_funcs: List, input_outputs: List[List[str]]):
    samples = [f"{input_output[0]} {input_output[1]}{EOS_TOKEN}" for input_output in input_outputs]

    inputs = [input_output[0] for input_output in input_outputs]
    outputs = [f"{input_output[1]}{EOS_TOKEN}" for input_output in input_outputs]
    expecteds = [input_output[2] for input_output in input_outputs]

    for sample, input, output, expected in zip(samples, inputs, outputs, expecteds):
        for reward_func in reward_funcs:
            kwargs = {
                'expected': [expected],
            }
            rewards_per_sample = reward_func([sample], [input], [output], tokenizer, **kwargs)
            for reward in rewards_per_sample:
                # title = f"{reward_func.__name__} | {reward}<br>Prompt: {input}<br>Correct: {output}"
                title = f"{reward_func.__name__}<br>Prompt: {input}<br>Correct: {output}"
                # print(title)

                tokens = tokenizer.tokenize(output)
                plot_token_rewards(tokens, reward, title=title)


In [65]:
from rewards import reward_basic, reward_basic_dense, make_reward_token_cost, make_reward_output_length_diff, reward_func_usage, make_reward_substring_matches, reward_func_usage_dense, make_reward_closing_brackets, make_reward_weighted_combo, penalize_syntax_errors

# reward_basic(["Input: 1 Output: [-4,-5,-2] Function: div_n(reverse([-2, -5, -4]),1)"])
input_outputs = [
    ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    # ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4])),1))", "div_n(reverse([-2, -5, -4]),1)"],
    # ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -4]))),1),),)", "div_n(reverse([-2, -5, -4]),1)"],
    # ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    # ["Input: 1 Output: [-4,-5,-2] Function:", "div(reverse([-2, -5, -4]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    # ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -a]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    ["Input: 1 Output: [-4,-5,-2] Function:", "div_n(reverse([-2, -5, -3]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    ["Input: 1 Output: [-4,-5,-2] Function:", "reverse(div_n(reverse(div_n([-2, -5, -3]),1)", "div_n(reverse([-2, -5, -4]),1)"],
    
    ["Input: 1 Output: [-4,-5,-2] Function:", " sort_asc(sort_asc(sort_asc(sort_asc(sort_asc(sort_asc([1] sort_asc([] sort_asc([] sort_asc([] sort_asc([] sort_asc([] sort_asc([]", "div_n(sort_asc([-2, -5, -4]),1)"],
    ["Input: 1 Output: [-4,-5,-2] Function:", "mul_users praying plunder([0, 4, -2, 0],5)", "mul(sort_asc([-2, -5, -4]),1)"],
    ["Input: 1 Output: [-4,-5,-2] Function:", "sort_asc(sort_asc(sort_asc(sort_asc(sort_asc(drop([1, -1, -2, -3],1)B-1))))", "mul(sort_asc([-2, -5, -4]),1)"],
    ["Input: 1 Output: [-4,-5,-2] Function:", "", "mul(sort_asc([-2, -5, -4]),1)"],
]

MAX_LENGTH = 200

compare_reward_funcs([
    # reward_basic,
    # reward_basic_dense,
    # make_reward_token_cost(MAX_LENGTH, 0.5, 0.01),
    # make_reward_token_cost(20, 0.5, 0.01),
    # make_reward_token_cost(10, 0.5, 0.01),
    # make_reward_output_length_diff(10),
    # make_reward_output_length_diff(),
    # reward_func_usage,
    # make_reward_substring_matches(["div_n(", "reverse("], 0.5),
    # make_reward_substring_matches(["["], 0.5),
    # make_reward_substring_matches(["[-2, -5, -4]"], 0.5),
    # reward_func_usage_dense,
    # make_reward_closing_brackets(max_reward=1.0),
    # penalize_syntax_errors,

    # make_reward_weighted_combo([
    #     # reward_basic,
    #     reward_basic_dense,
    #     make_reward_token_cost(MAX_LENGTH, 0.5, 0.01),
    #     reward_func_usage_dense,
    #     make_reward_closing_brackets(max_reward=1.0),
    #     make_reward_output_length_diff(),
    #     penalize_syntax_errors,
    # ], [
    #     1.0,
    #     0.1,
    #     0.3,
    #     0.2,
    #     0.1,
    #     0.5,
    # ]),

    make_reward_weighted_combo([
        reward_basic_dense,
        # make_reward_token_cost(MAX_LENGTH, 0.5, 0.01),
        make_reward_token_cost(MAX_LENGTH, 0.5, 0.01),
        reward_func_usage_dense,
        make_reward_closing_brackets(max_reward=1.0),
        make_reward_output_length_diff(),
        penalize_syntax_errors,
    ], [
        # 1.0,
        # 0.1,
        # 0.1,
        # 0.5,
        # 0.1,
        # 0.5,

        # Weights 2
        1.0,
        0.01,
        0.05,
        0.05,
        0.01,
        0.5,
    ])
], input_outputs)

'(' was never closed (<string>, line 1)
'(' was never closed (<string>, line 1)


invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
invalid syntax. Perhaps you forgot a comma? (<string>, line 1)


invalid syntax (<string>, line 1)
invalid syntax (<string>, line 1)


invalid syntax. Perhaps you forgot a comma? (<string>, line 1)
invalid syntax. Perhaps you forgot a comma? (<string>, line 1)


no overlapping tokens for (-1, -1) in 


In [9]:
text = "Your input text goes here. The quick brown fox jumps over the lazy dog!"
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
# clean_tokens = [f"'{token.replace('Ġ', ' ')}'" for token in tokens]

# let's assume rewards is a list of floats, where each float represents the reward of each token
# rewards = [0.2, -0.1, 0.5, -0.2, 0.1, 0.3, 0.0, 0.4]  # Modify according to your rewards
rewards = [0] * len(tokens)
# set a random value between -1 and 1 for each token
rewards = np.random.uniform(-1, 1, len(tokens))

print(tokens)
# plot_token_rewards(clean_tokens, rewards)
plot_token_rewards(tokens, rewards)

['Your', 'Ġinput', 'Ġtext', 'Ġgoes', 'Ġhere', '.', 'ĠThe', 'Ġquick', 'Ġbrown', 'Ġfox', 'Ġjumps', 'Ġover', 'Ġthe', 'Ġlazy', 'Ġdog', '!']


In [33]:
from lang import Interpreter
interpreter = Interpreter()

code = "mul_users prying plunder([0, 4, -2, 0],5)"
try:
    res = interpreter.eval(code)
    print(res)
except Exception as e:
    print(e)
    print(e.offset)
    print(code[e.offset - 1])
    print(code[e.end_offset - 1])

invalid syntax (<string>, line 1)
11
p
